In [1]:
# !pip install ipywidgets
# !pip install seaborn
# !pip install transformers
# from tqdm import tqdm
import sys
!{sys.executable} -m pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 880 kB 39.6 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=81e482e931c9cf9efb6451c9c32e499ba700a0561748c1b49341efd8c9563b3e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

from torch.utils.data import TensorDataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification

from matplotlib import pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import classification_report, confusion_matrix

# Get data (PNN dataset)

In [3]:
from google.colab import drive
import tensorflow as tf

drive.mount('/content/drive')
%cd drive/MyDrive/twitter_training/

df = pd.read_csv('./p_n_n_tilde_lv.csv')
df = df[['id', 'message_lv_tilde', 'label']]
df.head()

# df = pd.read_csv('./../../../labeledTweets/p_n_n_tilde_lv.csv')
# df = df[['id', 'message_lv_tilde', 'label']]
# df.head()

Mounted at /content/drive
/content/drive/MyDrive/twitter_training


,id,message_lv_tilde,label
0,1.340000e+18,@pilsonenjeff @lauferlaw @donwinslows pa reize...,negative
1,1.330000e+18,@tkdylan cilvēkiem ir aģentūra. Lins Vuds ir v...,negative
2,1.330000e+18,"@foenixaew Es nojaušu, ka WWE lika viņam iznāk...",negative
3,1.320000e+18,Maksvels droši vien pačurās mājā pirms mūsu nā...,negative
4,1.320000e+18,@msamson56 Esmu pārsteigts. KĀ cilvēki var atb...,negative


In [4]:
newLine ="\\n|\\r"
urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
numbers = '\d+((\.|\-)\d+)?'
mentions = '\B\@([\w\-]+)'
hashtag = '#'
whitespaces = '\s+'
leadTrailWhitespace = '^\s+|\s+?$'

df['clean_message'] = df['message_lv_tilde']
df['clean_message'] = df['clean_message'].str.replace(newLine,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(urls,' URL ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(mentions,' MENTION ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(numbers,' NMBR ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(hashtag,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(whitespaces,' ',regex=True)
df['clean_message'] = df['clean_message'].str.replace(leadTrailWhitespace,'',regex=True)

label_dict = {'neutral': 0, 'positive': 1, 'negative': 2}
df['label'] = df.label.replace(label_dict) 
df['label'].value_counts()

0    77028
1    51994
2    45622
Name: label, dtype: int64

In [5]:
df_0 = df[df['label']==0]
df_1 = df[df['label']==1]
df_2 = df[df['label']==2]

trainLabelSize = round(df_2.shape[0]*0.7)

df_0 = df_0.sample(trainLabelSize, random_state=42)
df_1 = df_1.sample(trainLabelSize, random_state=42)
df_2 = df_2.sample(trainLabelSize, random_state=42)

df_train = pd.concat([df_0, df_1, df_2])

df_train['label'].value_counts()

0    31935
1    31935
2    31935
Name: label, dtype: int64

# Add LV Twitter data for validation

In [6]:
df_lv = pd.read_csv('./allLabeledTweets.csv')
# df_lv = pd.read_csv('./../../../labeledTweets/allLabeledTweets.csv')
df_lv = df_lv[['id', 'message', 'label']]
df_lv = df_lv.drop_duplicates()

newLine ="\\n|\\r"
urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
numbers = '\d+((\.|\-)\d+)?'
mentions = '\B\@([\w\-]+)'
hashtag = '#'
whitespaces = '\s+'
leadTrailWhitespace = '^\s+|\s+?$'

df_lv['clean_message'] = df_lv['message']
df_lv['clean_message'] = df_lv['clean_message'].str.replace(newLine,' ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(urls,' URL ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(mentions,' MENTION ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(numbers,' NMBR ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(hashtag,' ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(whitespaces,' ',regex=True)
df_lv['clean_message'] = df_lv['clean_message'].str.replace(leadTrailWhitespace,'',regex=True)

df_0 = df_lv[df_lv['label']==0]
df_1 = df_lv[df_lv['label']==1]
df_2 = df_lv[df_lv['label']==2]

trainLabelSize_lv = round(df_1.shape[0]*0.85)

df_0 = df_0.sample(trainLabelSize_lv, random_state=42)
df_1 = df_1.sample(trainLabelSize_lv, random_state=42)
df_2 = df_2.sample(trainLabelSize_lv, random_state=42)

df_train_lv = pd.concat([df_0, df_1, df_2])
df_train = pd.concat([df_train, df_train_lv])
# Shuffle rows
# df_train = sklearn.utils.shuffle(df_train, random_state=42)

df_val = df_lv.merge(df_train_lv, on=['id', 'message', 'label', 'clean_message'], how='left', indicator=True)
df_val = df_val[df_val['_merge']=='left_only']
df_val = df_val[['id', 'message', 'label', 'clean_message']]

df_val['label'].value_counts()
# df = pd.concat([df, df_lv])
# df['label'].value_counts()

0    617
2    297
1     62
Name: label, dtype: int64

# Tokenizer "ELECTRA"

In [7]:
tokenizer = AutoTokenizer.from_pretrained('google/electra-base-discriminator', do_lower_case=True)

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

## Find popular UNK tokens

In [8]:
counter=[('🇱🇻', 7), ('👉', 6), ('🤔', 5), ('🎄', 5), ('😉', 4), ('😅', 3), ('😊', 3), ('❤️', 3), ('▶️', 3), ('😍', 2), ('🤗', 2), ('😀', 2), ('🙄', 2), ('🇷🇺', 2), ('🤷\u200d♀️', 2), ('🤬', 2), ('✨', 2), ('🙂', 2), ('📲', 2), ('✈️Izdevīgas', 1), ('laicīgi👍', 1), ('lidmašīnas😀😀😀', 1), ('👍🤗', 1), ('😥😥', 1), ('😵', 1), ('🇫🇷un', 1), ('nava😖', 1), ('nost!😁', 1), ('ŠVAMMES!😆', 1), ('🙊', 1), ('😁)', 1), ('😆', 1), ('🙈', 1), ('😁).', 1), ('defolts.😎😎😎😎😎', 1), ('📌', 1), ('🇷🇺🤥', 1), ('😂😂😂', 1), ('iekšā!🎥', 1), ('🤘🏼', 1), ('😂', 1), ('☝️', 1), ('cimdi.👍', 1), ('nesanāk🙁', 1), ('raibs....🤦\u200d♀️', 1), ('😁👍🏽', 1), ('🤣Interjera', 1), ('🤷🏼\u200d♂️🤦🏼\u200d♂️🕵🏼\u200d♂️😂', 1), ('🙄🙄🙄', 1), ('🍊', 1), ('🇱🇹', 1), ('😳', 1), ('🏅', 1), ('🎉', 1), ('🙌🏻', 1), ('atkarigs🤔', 1), ('👍🏻', 1), ('👍', 1), ('👏👏👏', 1), ('⭐️', 1), ('🚴\u200d♂️', 1), ('😂😂😂😂😂', 1), ('nenotiek😁', 1), ('stundu...😂', 1), ('rindas😉👍🏻', 1), ('risinājumiem.😂', 1), ('🤦\u200d♂️', 1), ('sarakstā?🤔', 1), ('darbību🤔tā', 1), ('☹', 1), ('🤔un', 1), ('reizi🙄', 1), ('😁😁😁🦄', 1), ('OK😁', 1), ('👌', 1), ('🤓', 1), ('pārvērtīsies!💥', 1), ('😌', 1), ('🤢🤑', 1), ('❗️', 1), ('🏎️', 1), ('☀️', 1), ('📸', 1), ('🕑', 1), ('⌚', 1), ('🖼️', 1), ('🎨👨\u200d🎨', 1), ('🏍', 1), ('jāmaina!🎶🚗', 1), ('🤘🔥', 1), ('🦁💡💪', 1), ('🎅', 1), ('🎄🎁', 1), ('🎬', 1), ('👇🎄', 1), ('vāciņu!📱🤳', 1), ('tagad!🎄', 1), ('👼', 1), ('😜', 1), ('❤', 1)]

most_common_values= [word for word, word_count in counter]

In [9]:
tokenizer.add_tokens(most_common_values, special_tokens=True)

100

### Find max length for tokenizer

In [10]:
token_lens = []
for txt in list(df.clean_message.values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_length = max(token_lens)
max_length

512

In [ ]:
max_length = 512

### Encode messages

In [11]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train["clean_message"].values.tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_val["clean_message"].values.tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',
    truncation=True,
    max_length=max_length, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

len(dataset_train), len(dataset_val)

(96849, 976)

# Model "ELECTRA"

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("google/electra-base-discriminator",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

In [13]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30622, 768)

In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 1
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batch_size)

In [15]:
from transformers import get_linear_schedule_with_warmup

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [16]:
epochs = 1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [17]:
# Function to measure weighted F1
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [18]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)
print(device)

cuda


# Load best model

In [19]:
model.load_state_dict(torch.load('./finetuned_ELECTRAbase_epoch_2.model', map_location=device))

<All keys matched successfully>

In [20]:
# Function to evaluate model. Returns average validation loss, predictions, true values
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    progress_bar = tqdm(dataloader_val, desc='Validating:', leave=False, disable=False)
    for batch in progress_bar:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

# Train

In [22]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += float(loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'ELECTRA_UNK_lowercase_pnn/finetuned_ELECTRA_epoch_{epoch}_pnn.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    preds_flat = np.argmax(predictions, axis=1).flatten()
    
    print('Classification report:')
    print(classification_report(true_vals, preds_flat))
    print('Confusion matrix:')
    print(pd.DataFrame(confusion_matrix(true_vals, preds_flat),
            index = [['actual', 'actual', 'actual'], ['neutral', 'positive', 'negative']],
            columns = [['predicted', 'predicted', 'predicted'], ['neutral', 'positive', 'negative']]))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/96849 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0650914652491406


Validating::   0%|          | 0/976 [00:00<?, ?it/s]

Validation loss: 1.0145645823146476
F1 Score (Weighted): 0.6148268633201845
Classification report:
              precision    recall  f1-score   support

           0       0.78      0.65      0.71       617
           1       0.10      0.37      0.15        62
           2       0.59      0.45      0.51       297

    accuracy                           0.57       976
   macro avg       0.49      0.49      0.46       976
weighted avg       0.68      0.57      0.61       976

Confusion matrix:
                predicted                  
                  neutral positive negative
actual neutral        403      134       80
       positive        25       23       14
       negative        86       78      133


# Evaluate

In [ ]:
model.load_state_dict(torch.load('X.model', map_location=device))

_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()

print(classification_report(true_vals, preds_flat))
print(pd.DataFrame(confusion_matrix(true_vals, preds_flat),
        index = [['actual', 'actual', 'actual'], ['neutral', 'positive', 'negative']],
        columns = [['predicted', 'predicted', 'predicted'], ['neutral', 'positive', 'negative']]))